In [1]:
from __future__ import print_function
"""
Tutorial: A reference implementation of configuration interactions singles.
"""

__authors__   = ["Boyi Zhang", "Adam S. Abbott"]
__credits__   = ["Boyi Zhang", "Adam S. Abbott", "Justin M. Turney"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-08-08"

# Configuration Interaction Singles (CIS) 

## I. Theoretical Overview

In this tutorial, we will implement the configuration interaction singles method in the spin orbital notation. The groundwork for working in the spin orbital notation has been laid out in "Introduction to the Spin Orbital Formulation of Post-HF methods" [tutorial](../08_CEPA0_and_CCD/8a_Intro_to_spin_orbital_postHF.ipynb). It is highly recommended to work through that introduction before starting this tutorial. 

### Configuration Interaction (CI)

The configuration interaction wavefunction is constructed as a linear combination of the reference determinants and all singly, doubly, ... n-tuple excited determinants where n is the number of electrons in a given system: 

\begin{equation}
\Psi_\mathrm{CI} = (1 + \hat{C_1} + \hat{C_2} + ...\hat{C_n)}\Phi
\end{equation}

Here, $\hat{C_n}$ is the n configuration excitation operator. 

In Full CI, all possible excitations are included in the wavefunction expansion. In truncated CI methods, only a subset of excitations are included. 

## CIS

In CIS, only single excitations from the occupied (indices i,j,k...) to the virtual (indices a,b,c...) orbitals are included. As a result, CIS gives transition energies to an excited state. 

Assuming we are using canonical Hartree-Fock spin orbitals($\{\mathrm{\psi_p}\}$) with orbital energies $\{\epsilon_p\}$, we can build a shifted CIS Hamiltonian matrix:

\begin{equation}
\tilde{\textbf{H}} = \textbf{H} - E_0 \textbf{I} = [\langle \Phi_P | \hat{H_e} - E_0|\Phi_Q \rangle],\, 
\Phi_P \in {\Phi_i^a}
\end{equation}

where $E_0$ is the ground state Hartree-Fock state energy given by $\langle \Phi | \hat{H_e}|\Phi \rangle$.

The matrix elements of this shifted CIS Hamiltonian matrix can be evaluated using Slater's rules to give:

\begin{equation}
\langle \Phi_i^a | \hat{H_e} - E_0|\Phi_j^b \rangle = (\epsilon_a - \epsilon_i)\delta_{ij} \delta_{ab}
+ \langle aj || ib \rangle
\end{equation}

This then becomes a standard eigenvalue equation from which we can solve for the excitation energies and the wavefunction expansion coefficients:

\begin{equation}
\tilde{\textbf{H}} \textbf{c}_K = \Delta E_K\textbf{c}_K, \,\Delta E_K = E_K - E_0
\end{equation}



## II. Implementation

As with previous tutorials, let's begin by importing Psi4 and NumPy and setting memory and output file options.

In [2]:
# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np

# ==> Set Basic Psi4 Options <==

# Memory specifications
psi4.set_memory(int(2e9))
numpy_memory = 2

# Output options
psi4.core.set_output_file('output.dat', False)

We now define the molecule and set Psi4 options:

In [3]:
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})


We use Psi4 to compute the RHF energy and wavefunction and store them in variables `scf_e and scf_wfn`. We also check the memory requirements for computation:

In [4]:
# Get the SCF wavefunction & energies
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)

# Check memory requirements
nmo = scf_wfn.nmo()
I_size = (nmo**4) * 8e-9
print('\nSize of the ERI tensor will be %4.2f GB.\n' % I_size)
memory_footprint = I_size * 1.5
if I_size > numpy_memory:
    psi4.core.clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds allotted \
                     memory limit of %4.2f GB." % (memory_footprint, numpy_memory))


Size of the ERI tensor will be 0.00 GB.



We first obtain orbital information from our wavefunction. We also create an instance of MintsHelper to help build our molecular integrals:

In [5]:
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(scf_wfn.basisset())

# Get basis and orbital information
nbf = mints.nbf()          # Number of basis functions
nalpha = scf_wfn.nalpha()  # Number of alpha electrons
nbeta = scf_wfn.nbeta()    # Number of beta electrons
nocc = nalpha + nbeta      # Total number of electrons
nso = 2 * nbf              # Total number of spin orbitals
nvirt = nso - nocc         # Number of virtual orbitals

We now build our 2-electron integral, a 4D tensor, in the spin orbital formulation. We also convert it into physicist's notation and antisymmetrize for easier manipulation of the tensor later on. 

In [6]:
def spin_block_tei(I):
    '''
    Spin blocks 2-electron integrals
    Using np.kron, we project I and I tranpose into the space of the 2x2 ide
    The result is our 2-electron integral tensor in spin orbital notation
     '''
    identity = np.eye(2)
    I = np.kron(identity, I)
    return np.kron(identity, I.T)
 
I = np.asarray(mints.ao_eri())
I_spinblock = spin_block_tei(I)
 
# Convert chemist's notation to physicist's notation, and antisymmetrize
# (pq | rs) ---> <pr | qs>
# <pr||qs> = <pr | qs> - <pr | sq>
gao = I_spinblock.transpose(0, 2, 1, 3) - I_spinblock.transpose(0, 2, 3, 1)


We get the orbital energies from alpha and beta electrons and append them together. We spin-block the coefficients obtained from the reference wavefunction and convert them into NumPy arrays. There is a set corresponding to coefficients from alpha electrons and a set of coefficients from beta electrons. We then sort them according to the order of the orbital energies using argsort():

In [7]:
# Get orbital energies, cast into NumPy array, and extend eigenvalues
eps_a = np.asarray(scf_wfn.epsilon_a())
eps_b = np.asarray(scf_wfn.epsilon_b())
eps = np.append(eps_a, eps_b)

# Get coefficients, block, and sort
Ca = np.asarray(scf_wfn.Ca())
Cb = np.asarray(scf_wfn.Cb())
C = np.block([
             [      Ca,         np.zeros_like(Cb)],
             [np.zeros_like(Ca),          Cb     ]])

# Sort the columns of C according to the order of orbital energies
C = C[:, eps.argsort()]

# Sort orbital energies
eps = np.sort(eps)


We now transform the 2-electron integral from the AO basis into the MO basis using the coefficients:


In [8]:
# Transform gao, which is the spin-blocked 4d array of physicist's notation,
# antisymmetric two-electron integrals, into the MO basis using MO coefficients
gmo = np.einsum('pQRS, pP -> PQRS',
      np.einsum('pqRS, qQ -> pQRS',
      np.einsum('pqrS, rR -> pqRS',
      np.einsum('pqrs, sS -> pqrS', gao, C), C), C), C)


Now that we have our integrals, coefficents, and orbital energies set up in with spin orbitals, we can start our CIS procedure. We first start by initializing the shifted Hamiltonion matrix $\tilde{\textbf{H}}$ (`HCIS`). Let's think about the size of $\tilde{\textbf{H}}$. We need all possible single excitations from the occupied to virtual orbitals. This is given by the number of occupied orbitals times the number of virtual orbitals  (`nocc * nvirt`).

The size of our matrix is thus `nocc * nvirt` by `nocc * nvirt`. 

In [9]:
# Initialize CIS matrix.
# The dimensions are the number of possible single excitations
HCIS = np.zeros((nocc * nvirt, nocc * nvirt))


Next, we want to build all possible excitations from occupied to virtual orbitals. We create two for-loops that will loop over the number of occupied orbitals and number of virtual orbitals, respectively, and store the combination of occupied and virtual indices as a tuple `(i, a)`. We put all tuples in a list called `excitations`. 

In [10]:
# Build the possible excitations, collect indices into a list
excitations = []
for i in range(nocc):
    for a in range(nocc, nso):
        excitations.append((i, a))

Now we can evaluate the matrix elements of the shifted CIS Hamiltonian matrix using the equation given above. For each element, there are several layers of indexing that we have to consider. 
First, there are the indices of the element itself, which gives the position of the element in the matrix. Indices `p` and `q` are used:

`HCIS[p, q]`

Second, there are two sets of excitations from occupied to virtual orbitals corresponding to the bra and ket of each matrix element. For these, we will take advantage of the `excitations` list that we build with the list of all possible excitations. We will use indices i and a to denote the excitation in the bra (`left_excitation`) and j and b to denote the excitation in the ket (`right_excitation`). 

To manage these indices, we will use the `enumerate` function.

Note that a Kronecker delta $\delta_{pq}$ can be represented as p == q. 


In [11]:
# Form matrix elements of shifted CIS Hamiltonian
for p, left_excitation in enumerate(excitations):
    i, a = left_excitation
    for q, right_excitation in enumerate(excitations):
        j, b = right_excitation
        HCIS[p, q] = (eps[a] - eps[i]) * (i == j) * (a == b) + gmo[a, j, i, b]


We now use the NumPy function `linalg.eigh` (for hermitian matrices) to diagonalize the shifted CIS Hamiltonian. This will give us the excitation energies (`ECIS`). These eigenvalues correspond to the CIS total energies for various excited states. The columns of matrix `CCIS` give us the coefficients which describe the relative contribution of each singly excited determinant to the excitation energy. 


In [12]:
# Diagonalize the shifted CIS Hamiltonian
ECIS, CCIS = np.linalg.eigh(HCIS)

For a given excitation energy, each coefficent in the linear combination of excitations represents the amount that a particular excitation contributes to the overall excitation energy. The percentage contribution of each coefficient can be calculated by squaring the coefficent and multiplying by 100. 

In [13]:
# Percentage contributions of coefficients for each state vector
percent_contrib = np.round(CCIS**2 * 100)


In addition to excitation energies, we want to print the excitations that contribute 10% or more to the overall energy, as well as their percent contribution. 

Note that `end = ''` parameter in the print function allows us to print different sections to the same line without a line break.

In [14]:
# Print detailed information on significant excitations
print('CIS:')
for state in range(len(ECIS)):
    # Print state, energy
    print('State %3d Energy (Eh) %10.7f' % (state + 1, ECIS[state]), end=' ')
    for idx, excitation in enumerate(excitations):
        if percent_contrib[idx, state] > 10:
            i, a = excitation
            # Print percentage contribution and the excitation
            print('%4d%% %2d -> %2d' % (percent_contrib[idx, state], i, a), end=' ')
    print()


CIS:
State   1 Energy (Eh)  0.2872554   99%  9 -> 10 
State   2 Energy (Eh)  0.2872554   99%  8 -> 11 
State   3 Energy (Eh)  0.2872554   50%  8 -> 10   50%  9 -> 11 
State   4 Energy (Eh)  0.3444249   44%  6 -> 10   44%  7 -> 11 
State   5 Energy (Eh)  0.3444249   88%  6 -> 11 
State   6 Energy (Eh)  0.3444249   88%  7 -> 10 
State   7 Energy (Eh)  0.3564617   50%  8 -> 10   50%  9 -> 11 
State   8 Energy (Eh)  0.3659889   95%  8 -> 13 
State   9 Energy (Eh)  0.3659889   95%  9 -> 12 
State  10 Energy (Eh)  0.3659889   50%  8 -> 12   50%  9 -> 13 
State  11 Energy (Eh)  0.3945137   35%  4 -> 11   51%  6 -> 13 
State  12 Energy (Eh)  0.3945137   20%  4 -> 10   20%  5 -> 11   30%  6 -> 12   30%  7 -> 13 
State  13 Energy (Eh)  0.3945137   35%  5 -> 10   51%  7 -> 12 
State  14 Energy (Eh)  0.4160717   50%  8 -> 12   50%  9 -> 13 
State  15 Energy (Eh)  0.5056282   44%  6 -> 10   44%  7 -> 11 
State  16 Energy (Eh)  0.5142899   51%  4 -> 11   36%  6 -> 13 
State  17 Energy (Eh)  0.514289

## References
1. Background paper:
 >"Toward a systematic molecular orbital theory for excited states"
[[Foresman:1992:96](http://pubs.acs.org/doi/abs/10.1021/j100180a030)] J. B. Foresman, M. Head-Gordon, J. A. Pople, M. J. Frisch, *J. Phys. Chem.* **96**, 135 (1992).


2. Algorithms from: 
	> [[CCQC:CIS](https://github.com/CCQC/summer-program/tree/master/7)] CCQC Summer Program, "CIS" accessed with https://github.com/CCQC/summer-program/tree/master/7.
    